In [7]:
import tensorflow as tf
!pip install -q -U "tensorflow-text==2.11.*"

!pip install cupy-cuda115
!pip install --upgrade numba 
!pip install --upgrade tensorflow==2.11
# !pip install numpy==1.22
!pip install gensim

!pip install -q tf-models-official==2.11

  Using cached numba-0.56.4-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.5 MB)
  Using cached llvmlite-0.39.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.6 MB)
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.38.0
ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [8]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [16]:
!pip install transformers


import pandas as pd
import torch
# from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup




import pandas as pd
import numpy as np
import matplotlib
# import seaborn as sns

# from gensim.models.word2vec import Word2Vec
import matplotlib.pyplot as plt
import pickle
import warnings

# !pip install nltk==3.5
# import nltk
# from nltk.stem import PorterStemmer #for stemming
# from nltk.tokenize import word_tokenize
# import nltk.corpus

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import  LogisticRegression
# from sklearn.linear_model import SGDClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import KFold
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import learning_curve
# from sklearn.metrics import confusion_matrix, f1_score, classification_report
# from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import average_precision_score
# from sklearn import svm



# !pip install 

warnings.filterwarnings("ignore")

In [ ]:
import tensorflow_models as tfm
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

ValueError: The name 'tf.TrackableConstant' has already been registered to a serializable class. Found: <class 'tensorflow.python.eager.function_saved_model_utils.TrackableConstant'>

In [3]:
path_to_train = "train.tsv"
path_to_test = "test.tsv"
path_to_valid = "valid.tsv"
df_train = pd.read_csv(path_to_train, delimiter='\t', dtype=object)
df_test = pd.read_csv(path_to_test, delimiter='\t', dtype=object)
df_valid = pd.read_csv(path_to_valid, delimiter='\t', dtype=object)

In [4]:
df_train.fillna("", inplace=True)
df_train.columns = [
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.
        
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement).
    ]


df_test.fillna("", inplace=True)
df_test.columns = [
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.
        
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement).
    ]

df_valid.fillna("", inplace=True)
df_valid.columns = [
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.
        
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement).
    ]

In [5]:
df_train = df_train.drop(columns=["id", "subjects", "speaker", "speaker_job_title", "state_info", "party_affiliation", "count_1", "count_2", "count_3", "count_4", "count_5", "context"])
df_train = df_train.replace({'label': {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}})
df_train = df_train.dropna()
train_labels = df_train["label"].values
train_sentences = df_train["statement"].values


df_valid = df_valid.drop(columns=["id", "subjects", "speaker", "speaker_job_title", "state_info", "party_affiliation", "count_1", "count_2", "count_3", "count_4", "count_5", "context"])
df_valid = df_valid.replace({'label': {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}})
df_valid = df_valid.dropna()
valid_labels = df_train["label"].values
valid_sentences = df_train["statement"].values

df_test = df_test.drop(columns=["id", "subjects", "speaker", "speaker_job_title", "state_info", "party_affiliation", "count_1", "count_2", "count_3", "count_4", "count_5", "context"])
df_test = df_test.replace({'label': {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}})
df_test = df_test.dropna()
test_labels = df_test["label"].values
test_sentences = df_test["statement"].values


In [6]:
# # Load BERT tokenizer and tokenize data
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# # Get encodings
# train_encodings = tokenizer(train_sentences.tolist(), truncation=True, padding=True)
# val_encodings = tokenizer(valid_sentences.tolist(), truncation=True, padding=True)
# test_encodings = tokenizer(test_sentences.tolist(), truncation=True, padding=True)

In [9]:
with tf.device('/cpu:0'):
  train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels))
  valid_dataset = tf.data.Dataset.from_tensor_slices((valid_sentences, valid_sentences))

2023-04-26 09:16:05.840879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 09:16:05.892755: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 09:16:05.894430: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 09:16:05.897140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [10]:
# # Convert data to PyTorch tensors
# train_inputs = {key: torch.tensor(val) for key, val in train_encodings.items()}
# train_labels = torch.tensor(train_labels)
# val_inputs = {key: torch.tensor(val) for key, val in val_encodings.items()}
# val_labels = torch.tensor(val_labels)

In [11]:
# ########Convert to tf tensors

# import tensorflow as tf

# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     train_labels
# ))

# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(test_encodings),
#     test_labels
# ))


# # valid_dataset = tf.data.Dataset.from_tensor_slices((
# #     dict(val_encodings),
# #     valid_labels
# # ))


# valid_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_encodings)
# ))

In [13]:
# # Define model
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6, output_attentions=False, output_hidden_states=False)

In [14]:
# # Define optimizer and learning rate
# optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
# num_epochs = 4
# num_training_steps = num_epochs * len(train_inputs)
# num_warmup_steps = int(0.1 * num_training_steps)
# lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

In [15]:
# # Train
# model.cuda()
# model.train()
# for epoch in range(num_epochs):
#     for i in range(0, len(train_inputs), 32):
#         optimizer.zero_grad()
#         batch_inputs = {key: val[i:i+32].cuda() for key, val in train_inputs.items()}
#         batch_labels = train_labels[i:i+32].cuda()
#         outputs = model(**batch_inputs, labels=batch_labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         lr_scheduler.step()

#     # Evaluate on validation set
#     model.eval()
#     val_loss = 0
#     val_correct = 0
#     with torch.no_grad():
#         for i in range(0, len(val_inputs), 32):
#             batch_inputs = {key: val[i:i+32].cuda() for key, val in val_inputs.items()}
#             batch_labels = val_labels[i:i+32].

In [16]:
# from transformers import TFBertForSequenceClassification


# # Fine-tune the pre-trained BERT model on the training data
# model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
# model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
# history = model.fit(train_dataset.batch(16), epochs=5, validation_data=valid_dataset.batch(16))

# # Plot the training and validation loss
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']
# epochs = range(1, len(train_loss) + 1)

# plt.plot(epochs, train_loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# # Evaluate the fine-tuned model on the test data
# test_predictions = model.predict(test_dataset.batch(16)).logits
# test_predictions = tf.nn.softmax(test_predictions, axis=1).numpy()
# test_predicted_labels = test_predictions.argmax(axis=1)

In [17]:
from transformers import TFBertForSequenceClassification

In [18]:
# def binary_crossentropy_with_logits(labels, logits):
#     labels = tf.cast(labels, tf.float32)
#     return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits))


In [ ]:
# Load BERT tokenizer and tokenize data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [19]:
# Load the BERT model

num_classes = 6
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=num_classes)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
label_list = [0, 1,2,3,4,5]
max_seq_length =  128
batch_size=32


from official.nlp.bert import tokenization
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

ModuleNotFoundError: No module named 'official.nlp.bert'

In [ ]:
tokenizer = tfm.nlp.layers.FastWordpieceBertTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
    lower_case=True)

In [21]:
# This provides a function to convert row to input features and label

def to_feature(text, label, label_list=label_list, max_seq_length=max_seq_length, tokenizer=tokenizer):
  example = classifier_data_lib.InputExample(guid = None,
                                            text_a = text.numpy(), 
                                            text_b = None, 
                                            label = label.numpy())
  feature = classifier_data_lib.convert_single_example(0, example, label_list,
                                    max_seq_length, tokenizer)
  
  return (feature.input_ids, feature.input_mask, feature.segment_ids, feature.label_id)

NameError: name 'tokenizer' is not defined

In [67]:
def to_feature_map(text, label):
  input_ids, input_mask, segment_ids, label_id = tf.py_function(to_feature, inp=[text, label], 
                                Tout=[tf.int32, tf.int32, tf.int32, tf.int32])

  # py_func doesn't set the shape of the returned tensors.
  input_ids.set_shape([max_seq_length])
  input_mask.set_shape([max_seq_length])
  segment_ids.set_shape([max_seq_length])
  label_id.set_shape([])

  x = {
        'input_word_ids': input_ids,
        'input_mask': input_mask,
        'input_type_ids': segment_ids
    }
  return (x, label_id)


In [68]:
# train_dataset = train_dataset.shuffle(len(train_sentences)).batch(8).cache().prefetch(tf.data.AUTOTUNE)
# test_dataset = test_dataset.batch(8).cache().prefetch(tf.data.AUTOTUNE)
# valid_dataset = valid_dataset.batch(8).cache().prefetch(tf.data.AUTOTUNE)

In [69]:
with tf.device('/cpu:0'):
  # train
  train_dataset = (train_dataset.map(to_feature_map,
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
                          #.cache()
                          .shuffle(1000)
                          .batch(32, drop_remainder=True)
                          .prefetch(tf.data.experimental.AUTOTUNE))

  # valid
  valid_dataset = (valid_dataset.map(to_feature_map,
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)
                          .batch(32, drop_remainder=True)
                          .prefetch(tf.data.experimental.AUTOTUNE)) 

TypeError: in user code:

    File "/tmp/ipykernel_2722/3975330603.py", line 2, in to_feature_map  *
        input_ids, input_mask, segment_ids, label_id = tf.py_function(to_feature, inp=[text, label],

    TypeError: Tensors in list passed to 'input' of 'EagerPyFunc' Op have types [<NOT CONVERTIBLE TO TENSOR>, int64] that are invalid. Tensors: [{'input_ids': <tf.Tensor 'args_1:0' shape=(512,) dtype=int32>, 'token_type_ids': <tf.Tensor 'args_2:0' shape=(512,) dtype=int32>, 'attention_mask': <tf.Tensor 'args_0:0' shape=(512,) dtype=int32>}, <tf.Tensor 'args_3:0' shape=() dtype=int64>]


In [47]:

device = tf.device('/gpu:0')

# bert_model.compile(optimizer=optimizer, loss=binary_crossentropy_with_logits, metrics=['accuracy'])

# Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
# bert_model.layers[-1].activation = tf.keras.activations.softmax
# bert_model.compile(optimizer=optimizer, loss=tf.nn.sigmoid_cross_entropy_with_logits, metrics=['accuracy'])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-08, clipnorm=1.0)
bert_model.compile(optimizer=optimizer, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])



# Train the model
history = bert_model.fit(train_dataset, epochs=5, validation_data=valid_dataset)

# Plot the training and validation loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/5


ValueError: in user code:

    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.8/site-packages/transformers/modeling_tf_utils.py", line 1567, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filei7mjfbdb.py", line 36, in tf__run_call_with_unpacked_inputs
        retval_ = ag__.converted_call(ag__.ld(func), (ag__.ld(self),), dict(**ag__.